In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
import seaborn as sns

['train (1).csv', 'test_bqCt9Pv.csv', 'submission (1).csv']


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#%%capture
#!pip install fastai==0.7.0
#from fastai.imports import *
#from fastai.structured import *

In [4]:
#from fastai.imports import *
#from fastai.structured import *

from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
# reading the data

train = pd.read_csv('../input/train (1).csv', parse_dates=["Date.of.Birth", "DisbursalDate"])
test = pd.read_csv('../input/test_bqCt9Pv.csv', parse_dates=["Date.of.Birth", "DisbursalDate"])
sub = pd.read_csv('../input/submission (1).csv')

# getting the shapes of the datasets
print("Shape of Train :", train.shape)
print("Shape of Test :", test.shape)
print("Shape of submission :", sub.shape)

Shape of Train : (233154, 41)
Shape of Test : (112392, 40)
Shape of submission : (112392, 2)


In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
display_all(train[5:100])

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
5,518279,54513,61900,89.66,67,22807,45,1501,1990-08-09,Self employed,2018-09-19,6,1998,1,1,0,0,0,0,825,A-Very Low Risk,2,0,0,0,0,0,0,0,0,0,0,0,1347,0,0,0,1yrs 9mon,2yrs 0mon,0,0
6,529269,46349,61500,76.42,67,22807,45,1502,1988-01-06,Salaried,2018-09-23,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
7,510278,43894,61900,71.89,67,22807,45,1501,1989-04-10,Salaried,2018-09-16,6,1998,1,1,0,0,0,0,17,Not Scored: Not Enough Info available on the c...,1,1,0,72879,74500,74500,0,0,0,0,0,0,0,0,0,0,0yrs 2mon,0yrs 2mon,0,0
8,490213,53713,61973,89.56,67,22807,45,1497,1991-11-15,Self employed,2018-05-09,6,1998,1,1,0,0,0,0,718,D-Very Low Risk,1,1,0,-41,365384,365384,0,0,0,0,0,0,0,0,0,0,4yrs 8mon,4yrs 8mon,1,0
9,510980,52603,61300,86.95,67,22807,45,1492,2068-01-06,Salaried,2018-09-16,6,1998,1,0,0,1,0,0,818,A-Very Low Risk,1,0,0,0,0,0,0,0,0,0,0,0,2608,0,0,0,1yrs 7mon,1yrs 7mon,0,0
10,548567,53278,61230,89.83,67,22807,45,1493,1979-01-01,Self employed,2018-09-29,6,1998,1,1,0,0,0,0,300,M-Very High Risk,1,0,0,0,0,0,0,0,0,0,0,0,2270,0,0,0,0yrs 7mon,0yrs 7mon,0,1
11,486821,64769,74190,89.23,67,22807,45,1446,1984-07-09,Salaried,2018-03-09,6,1998,1,1,0,0,0,0,786,B-Very Low Risk,3,2,0,676,36154,23374,0,0,0,0,0,0,0,0,0,0,2yrs 1mon,2yrs 3mon,1,0
12,478647,53278,61330,89.68,67,22807,45,1497,1974-01-06,Salaried,2018-08-30,6,1998,1,0,0,1,0,0,300,M-Very High Risk,7,2,1,0,69900,69900,0,0,0,0,0,0,3300,0,0,0,1yrs 3mon,2yrs 9mon,0,1
13,479533,49478,57010,89.46,67,22807,45,1497,1984-08-16,Salaried,2018-08-30,6,1998,1,1,0,0,0,0,738,C-Very Low Risk,10,5,0,79750,187000,187000,0,0,0,0,0,0,23309,0,1,0,1yrs 0mon,2yrs 1mon,4,1
14,483869,49278,57080,89.35,67,22807,45,1495,1973-02-18,Self employed,2018-08-31,6,1998,1,1,0,0,0,0,300,M-Very High Risk,5,5,3,95597,179252,179252,0,0,0,0,0,0,3514,0,0,0,3yrs 11mon,7yrs 2mon,0,1


In [8]:
unique_id_train = train['UniqueID']
unique_id_test = test['UniqueID']
data = pd.concat([train, test], axis = 0, ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
data['Disbursal_month'] = data['DisbursalDate'].dt.month

In [10]:
display_all(data.isnull().sum().sort_index()/len(data))
data.isnull().sum()

AVERAGE.ACCT.AGE                       0.000000
Aadhar_flag                            0.000000
CREDIT.HISTORY.LENGTH                  0.000000
Current_pincode_ID                     0.000000
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS    0.000000
Date.of.Birth                          0.000000
DisbursalDate                          0.000000
Disbursal_month                        0.000000
Driving_flag                           0.000000
Employee_code_ID                       0.000000
Employment.Type                        0.032135
MobileNo_Avl_Flag                      0.000000
NEW.ACCTS.IN.LAST.SIX.MONTHS           0.000000
NO.OF_INQUIRIES                        0.000000
PAN_flag                               0.000000
PERFORM_CNS.SCORE                      0.000000
PERFORM_CNS.SCORE.DESCRIPTION          0.000000
PRI.ACTIVE.ACCTS                       0.000000
PRI.CURRENT.BALANCE                    0.000000
PRI.DISBURSED.AMOUNT                   0.000000
PRI.NO.OF.ACCTS                        0

AVERAGE.ACCT.AGE                            0
Aadhar_flag                                 0
CREDIT.HISTORY.LENGTH                       0
Current_pincode_ID                          0
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS         0
Date.of.Birth                               0
DisbursalDate                               0
Driving_flag                                0
Employee_code_ID                            0
Employment.Type                         11104
MobileNo_Avl_Flag                           0
NEW.ACCTS.IN.LAST.SIX.MONTHS                0
NO.OF_INQUIRIES                             0
PAN_flag                                    0
PERFORM_CNS.SCORE                           0
PERFORM_CNS.SCORE.DESCRIPTION               0
PRI.ACTIVE.ACCTS                            0
PRI.CURRENT.BALANCE                         0
PRI.DISBURSED.AMOUNT                        0
PRI.NO.OF.ACCTS                             0
PRI.OVERDUE.ACCTS                           0
PRI.SANCTIONED.AMOUNT             

In [11]:
#display_all(data.isnull().sum().sort_index()/len(data))
#data.isnull().sum()
"""data['asset_cost_flag'] = 0 #eligible
data['asset_cost_flag'][data['asset_cost']>200000] = 1
data['asset_cost_flag'][data['asset_cost']<40000] = 1
data['asset_cost_flag'].value_counts()"""

"data['asset_cost_flag'] = 0 #eligible\ndata['asset_cost_flag'][data['asset_cost']>200000] = 1\ndata['asset_cost_flag'][data['asset_cost']<40000] = 1\ndata['asset_cost_flag'].value_counts()"

In [12]:
data['disbursed_amount_flag'] = 0 # eligible
data['disbursed_amount_flag'][data['disbursed_amount']<8000]=1 # not eligible
data['disbursed_amount_flag'][data['disbursed_amount']>200000] = 1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
data['Employment.Type'].fillna(1, inplace = True)# not eligible
data['Employment.Type'] = data['Employment.Type'].replace('Self employed', 0)
data['Employment.Type'] = data['Employment.Type'].replace('Salaried', 0)
#data['Employment.Type'] = data['Employment.Type'].replace('Unemployed', 1) 
data['Employment.Type'].value_counts()

0    334442
1     11104
Name: Employment.Type, dtype: int64

In [14]:
#Age of the customer at the time of loan disbursal
# extracting the year of birth of the customers
data['Year_of_birth'] = data['Date.of.Birth'].dt.year
data['Age']=2018-data['Year_of_birth']
#converting ALL negative ages to zero
data['Age'][data['Age'] < 0] = 0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
#calculating remaining time to retirement 
data['time.to.retire']= 65 - data['Age']
data['time.to.retire'][data['time.to.retire'] < 0] = 0
data['time.to.retire'][data['time.to.retire'] > 47] = 0
data['within.age.limit'] = 1 #eligible for loan
data['within.age.limit'][data['Age'] < 18] = 0
data['within.age.limit'][data['Age'] > 65] = 0
data['within.age.limit'].value_counts()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ip

1    315841
0     29705
Name: within.age.limit, dtype: int64

In [16]:
# encodings for bureau score(perform cns score distribution)

data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('No Bureau History Available', 0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Sufficient History Not Available', 0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Not Enough Info available on the customer', 0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Activity seen on the customer (Inactive)',0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Updates available in last 36 months', 0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Only a Guarantor', 0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: More than 50 active Accounts found',0)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('M-Very High Risk', 5)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('L-Very High Risk', 5)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('K-High Risk', 4)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('J-High Risk', 4)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('I-Medium Risk', 3)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('H-Medium Risk', 3)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('G-Low Risk', 2)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('F-Low Risk', 2)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('E-Low Risk', 2)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('D-Very Low Risk', 1)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('C-Very Low Risk', 1)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('B-Very Low Risk', 1)
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('A-Very Low Risk', 1)

# checing the values in bureau score
data['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

0    192802
1     75721
2     26364
3     18402
4     17667
5     14590
Name: PERFORM_CNS.SCORE.DESCRIPTION, dtype: int64

In [17]:
data['manufacturer_id'] = data['manufacturer_id'].replace(86, 11)
data['manufacturer_id'] = data['manufacturer_id'].replace(45, 10)
data['manufacturer_id'] = data['manufacturer_id'].replace(51, 9)
data['manufacturer_id'] = data['manufacturer_id'].replace(48, 8)
data['manufacturer_id'] = data['manufacturer_id'].replace(49, 7)
data['manufacturer_id'] = data['manufacturer_id'].replace(120, 6)
data['manufacturer_id'] = data['manufacturer_id'].replace(67, 5)
data['manufacturer_id'] = data['manufacturer_id'].replace(145, 4)
data['manufacturer_id'] = data['manufacturer_id'].replace(153, 3)
data['manufacturer_id'] = data['manufacturer_id'].replace(152, 2)
data['manufacturer_id'] = data['manufacturer_id'].replace(156, 1)
data['manufacturer_id'] = data['manufacturer_id'].replace(155, 0)
data['manufacturer_id'].value_counts()

11    161203
10     87053
9      40927
8      22964
7      14812
6      14049
5       3364
4       1138
3         25
2          9
1          1
0          1
Name: manufacturer_id, dtype: int64

In [18]:
data['Postal.zone'] = round(data['Current_pincode_ID']/1000)
data['Postal.zone'] = data['Postal.zone'].astype(int)
data['major_city'] = 1 # not a major city
data['major_city'][data['Postal.zone']==1] = 0
data['major_city'][data['Postal.zone']==4] = 0
data['major_city'][data['Postal.zone']==6] = 0
data['major_city'][data['Postal.zone']==7] = 0
data['major_city'].value_counts()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set 

1    182514
0    163032
Name: major_city, dtype: int64

In [19]:
#Converting the AVERAGE.ACCT.AGE to months
data['AVERAGE.ACCT.AGE_year'] = data['AVERAGE.ACCT.AGE'].apply(lambda x: x.split('yrs')[0])
data['AVERAGE.ACCT.AGE_month'] = data['AVERAGE.ACCT.AGE'].apply(lambda x: x.split(' ')[1])
data['AVERAGE.ACCT.AGE_month'] = data['AVERAGE.ACCT.AGE_month'].apply(lambda x: x.split('mon')[0])
data['AVERAGE.ACCT.AGE_month']= data['AVERAGE.ACCT.AGE_month'].astype(int)
data['AVERAGE.ACCT.AGE_year'] = data['AVERAGE.ACCT.AGE_year'].astype(int)
data['AVERAGE.ACCT.AGE_period']= round((data['AVERAGE.ACCT.AGE_year']*12 + data['AVERAGE.ACCT.AGE_month'])/12,1)

#Converting the CREDIT.HISTORY.LENGTH to years
data['CREDIT.HISTORY.LENGTH_year'] = data['CREDIT.HISTORY.LENGTH'].apply(lambda x: x.split('yrs')[0])
data['CREDIT.HISTORY.LENGTH_month'] = data['CREDIT.HISTORY.LENGTH'].apply(lambda x: x.split(' ')[1])
data['CREDIT.HISTORY.LENGTH_month'] = data['CREDIT.HISTORY.LENGTH_month'].apply(lambda x: x.split('mon')[0])
data['CREDIT.HISTORY.LENGTH_month']= data['CREDIT.HISTORY.LENGTH_month'].astype(int)
data['CREDIT.HISTORY.LENGTH_year'] = data['CREDIT.HISTORY.LENGTH_year'].astype(int)
data['CREDIT.HISTORY.LENGTH_period']= round((data['CREDIT.HISTORY.LENGTH_year']*12 + data['CREDIT.HISTORY.LENGTH_month'])/12,1)
data['CREDIT.HISTORY.LENGTH_period'].value_counts()
data['AVERAGE.ACCT.AGE_period']

0         0.0
1         1.9
2         0.0
3         0.7
4         0.0
5         1.8
6         0.0
7         0.2
8         4.7
9         1.6
10        0.6
11        2.1
12        1.2
13        1.0
14        3.9
15        0.1
16        2.5
17        1.8
18        1.8
19        0.9
20        0.5
21        2.8
22        0.0
23        1.2
24        0.0
25        0.0
26        0.0
27        0.0
28        3.1
29        2.0
         ... 
345516    0.2
345517    0.0
345518    1.8
345519    3.6
345520    0.0
345521    0.0
345522    0.0
345523    0.5
345524    0.2
345525    0.0
345526    0.0
345527    0.0
345528    0.8
345529    0.0
345530    0.0
345531    0.8
345532    0.0
345533    0.0
345534    0.0
345535    0.8
345536    0.0
345537    3.8
345538    0.0
345539    0.0
345540    1.1
345541    0.0
345542    0.0
345543    4.8
345544    0.2
345545    0.0
Name: AVERAGE.ACCT.AGE_period, Length: 345546, dtype: float64

In [20]:
data['loan_tenure_flag'] = 0
data['loan_tenure_flag'][data['AVERAGE.ACCT.AGE_period']<0.25]=1 #not valid
data['loan_tenure_flag'][data['AVERAGE.ACCT.AGE_period']>4]=1 # not valid
data['loan_tenure_flag'].value_counts()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1    199249
0    146297
Name: loan_tenure_flag, dtype: int64

In [21]:
#data.drop(['VoterID_flag', 'PAN_flag', 'Driving_flag'], axis = 1, inplace = True)

In [22]:
data['TOTAL.NO.OF.ACCTS'] = data['PRI.NO.OF.ACCTS'] + data['SEC.NO.OF.ACCTS']
data['TOTAL.ACTIVE.ACCTS'] = data['PRI.ACTIVE.ACCTS'] + data['SEC.ACTIVE.ACCTS']
data['TOTAL.OVERDUE.ACCTS'] = data['PRI.OVERDUE.ACCTS'] + data['SEC.OVERDUE.ACCTS']
data['TOTAL.CURRENT.BALANCE'] = data['PRI.CURRENT.BALANCE'] + data['SEC.CURRENT.BALANCE']
data['TOTAL.SANCTIONED.AMOUNT'] = data['PRI.SANCTIONED.AMOUNT'] +data['SEC.SANCTIONED.AMOUNT']
data['TOTAL.DISBURSED.AMOUNT'] = data['PRI.DISBURSED.AMOUNT'] + data['SEC.DISBURSED.AMOUNT']
data['TOTAL.INSTAL.AMT'] = data['PRIMARY.INSTAL.AMT'] + data['SEC.INSTAL.AMT']
data.drop(['PRI.NO.OF.ACCTS','PRI.ACTIVE.ACCTS',  'PRI.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT',
           'PRI.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.NO.OF.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
           'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT','SEC.ACTIVE.ACCTS', 'SEC.INSTAL.AMT'], axis = 1, inplace = True)


In [23]:
data['TOTAL.CURRENT.BALANCE'][data['TOTAL.CURRENT.BALANCE']<0]=0
data['TOTAL.SANCTIONED.AMOUNT'][data['TOTAL.SANCTIONED.AMOUNT']<0]=0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
print(data.shape)
print(data.columns)

(345546, 49)
Index(['AVERAGE.ACCT.AGE', 'Aadhar_flag', 'CREDIT.HISTORY.LENGTH',
       'Current_pincode_ID', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'Date.of.Birth', 'DisbursalDate', 'Driving_flag', 'Employee_code_ID',
       'Employment.Type', 'MobileNo_Avl_Flag', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'NO.OF_INQUIRIES', 'PAN_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'Passport_flag', 'State_ID',
       'UniqueID', 'VoterID_flag', 'asset_cost', 'branch_id',
       'disbursed_amount', 'loan_default', 'ltv', 'manufacturer_id',
       'supplier_id', 'Disbursal_month', 'disbursed_amount_flag',
       'Year_of_birth', 'Age', 'time.to.retire', 'within.age.limit',
       'Postal.zone', 'major_city', 'AVERAGE.ACCT.AGE_year',
       'AVERAGE.ACCT.AGE_month', 'AVERAGE.ACCT.AGE_period',
       'CREDIT.HISTORY.LENGTH_year', 'CREDIT.HISTORY.LENGTH_month',
       'CREDIT.HISTORY.LENGTH_period', 'loan_tenure_flag', 'TOTAL.NO.OF.ACCTS',
       'TOTAL.ACTIVE.ACCTS', 'TOT

In [25]:
data.drop(['Date.of.Birth','Employee_code_ID','UniqueID','supplier_id', 'Year_of_birth', 'Current_pincode_ID', 'MobileNo_Avl_Flag', 'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH','AVERAGE.ACCT.AGE_year', 'AVERAGE.ACCT.AGE_month','CREDIT.HISTORY.LENGTH_year',
           'CREDIT.HISTORY.LENGTH_month', 'DisbursalDate' ],axis = 1, inplace = True) 

In [26]:
#data.drop(['Date.of.Birth','disbursed_amount','asset_cost', 'branch_id','Employee_code_ID','State_ID', 'UniqueID', 'Year_of_birth', 'Age', 'Current_pincode_ID', 'supplier_id', 'MobileNo_Avl_Flag', 'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH','AVERAGE.ACCT.AGE_year', 'AVERAGE.ACCT.AGE_month','CREDIT.HISTORY.LENGTH_year','CREDIT.HISTORY.LENGTH_month', 'DisbursalDate','PERFORM_CNS.SCORE' ]
                 #, axis = 1, inplace = True)
print(data.shape)
print(data.columns)

(345546, 35)
Index(['Aadhar_flag', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'Driving_flag',
       'Employment.Type', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
       'PAN_flag', 'PERFORM_CNS.SCORE', 'PERFORM_CNS.SCORE.DESCRIPTION',
       'Passport_flag', 'State_ID', 'VoterID_flag', 'asset_cost', 'branch_id',
       'disbursed_amount', 'loan_default', 'ltv', 'manufacturer_id',
       'Disbursal_month', 'disbursed_amount_flag', 'Age', 'time.to.retire',
       'within.age.limit', 'Postal.zone', 'major_city',
       'AVERAGE.ACCT.AGE_period', 'CREDIT.HISTORY.LENGTH_period',
       'loan_tenure_flag', 'TOTAL.NO.OF.ACCTS', 'TOTAL.ACTIVE.ACCTS',
       'TOTAL.OVERDUE.ACCTS', 'TOTAL.CURRENT.BALANCE',
       'TOTAL.SANCTIONED.AMOUNT', 'TOTAL.DISBURSED.AMOUNT',
       'TOTAL.INSTAL.AMT'],
      dtype='object')


In [27]:
data['branch_id'] = data['branch_id'].astype('category')#82 unique
data['State_ID'] = data['State_ID'].astype('category')#22 unique
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['branch_id'] = le.fit_transform(data['branch_id'])
data['State_ID'] = le.fit_transform(data['State_ID'])

In [28]:
#display_all(data)

skewness measure

In [29]:
data['TOTAL.ACTIVE.ACCTS'] = np.log1p(data['TOTAL.ACTIVE.ACCTS'])
data['TOTAL.CURRENT.BALANCE'] = np.log1p(data['TOTAL.CURRENT.BALANCE'])
data['TOTAL.DISBURSED.AMOUNT']= np.log1p(data['TOTAL.DISBURSED.AMOUNT'])
data['TOTAL.INSTAL.AMT']= np.log1p(data['TOTAL.INSTAL.AMT'])
data['TOTAL.NO.OF.ACCTS']= np.log1p(data['TOTAL.NO.OF.ACCTS'])
data['TOTAL.OVERDUE.ACCTS']= np.log1p(data['TOTAL.OVERDUE.ACCTS'])
data['TOTAL.SANCTIONED.AMOUNT']= np.log1p(data['TOTAL.SANCTIONED.AMOUNT'])

In [30]:

"""ltv =data['ltv'].median()
cred  = data['CREDIT.HISTORY.LENGTH_period'].median()
accage = data['AVERAGE.ACCT.AGE_period'].median()
newacc = data['NEW.ACCTS.IN.LAST.SIX.MONTHS'].median()
asset=data['asset_cost'].median()
disbamt = data['disbursed_amount'].median()
print(ltv, cred, accage, newacc, asset, disbamt)"""

"ltv =data['ltv'].median()\ncred  = data['CREDIT.HISTORY.LENGTH_period'].median()\naccage = data['AVERAGE.ACCT.AGE_period'].median()\nnewacc = data['NEW.ACCTS.IN.LAST.SIX.MONTHS'].median()\nasset=data['asset_cost'].median()\ndisbamt = data['disbursed_amount'].median()\nprint(ltv, cred, accage, newacc, asset, disbamt)"

In [31]:
"""data['CREDIT.HISTORY.LENGTH_period']= np.log1p(data['CREDIT.HISTORY.LENGTH_period'])
data['AVERAGE.ACCT.AGE_period']= np.log1p(data['AVERAGE.ACCT.AGE_period'])
data['NEW.ACCTS.IN.LAST.SIX.MONTHS']= np.log1p(data['NEW.ACCTS.IN.LAST.SIX.MONTHS'])
data['asset_cost'] = np.log1p(data['asset_cost'])
data['disbursed_amount']=np.log1p(data['disbursed_amount'])
data['ltv']=np.log1p(data['ltv'])"""

"data['CREDIT.HISTORY.LENGTH_period']= np.log1p(data['CREDIT.HISTORY.LENGTH_period'])\ndata['AVERAGE.ACCT.AGE_period']= np.log1p(data['AVERAGE.ACCT.AGE_period'])\ndata['NEW.ACCTS.IN.LAST.SIX.MONTHS']= np.log1p(data['NEW.ACCTS.IN.LAST.SIX.MONTHS'])\ndata['asset_cost'] = np.log1p(data['asset_cost'])\ndata['disbursed_amount']=np.log1p(data['disbursed_amount'])\ndata['ltv']=np.log1p(data['ltv'])"

In [32]:
#data['TOTAL.CURRENT.BALANCE'].fillna(0, inplace = True)
#data['TOTAL.SANCTIONED.AMOUNT'].fillna(0, inplace = True)
data.isnull().sum()

Aadhar_flag                                 0
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS         0
Driving_flag                                0
Employment.Type                             0
NEW.ACCTS.IN.LAST.SIX.MONTHS                0
NO.OF_INQUIRIES                             0
PAN_flag                                    0
PERFORM_CNS.SCORE                           0
PERFORM_CNS.SCORE.DESCRIPTION               0
Passport_flag                               0
State_ID                                    0
VoterID_flag                                0
asset_cost                                  0
branch_id                                   0
disbursed_amount                            0
loan_default                           112392
ltv                                         0
manufacturer_id                             0
Disbursal_month                             0
disbursed_amount_flag                       0
Age                                         0
time.to.retire                    

In [33]:
#split into original test and train
data['loan_default'].fillna(2, inplace = True)
test_df = data[data['loan_default'] == 2]
train_df = data[data['loan_default'] != 2]
test_df.drop(['loan_default'], axis = 1, inplace = True)
print("train shape" + str(train_df.shape))
print("test shape"+str(test_df.shape))

train shape(233154, 35)
test shape(112392, 34)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [34]:
#train_df['loan_default'][train_df['within.age.limit']==0] = 1#not eligible for loan
#train_df['loan_default'][train_df['Employment.Type']==0] = 1
# 1 = loan defaulty and 0 non loan defaultys
y_train = train_df['loan_default']
train_df.drop(['loan_default'], axis = 1, inplace = True)
y_train.value_counts()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


0.0    182543
1.0     50611
Name: loan_default, dtype: int64

In [35]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /opt/conda/lib/python3.6/site-packages/imbalanced_learn-0.5.0.dev0-py3.6.egg (0.5.0.dev0)


In [36]:
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [37]:
from imblearn.over_sampling import SMOTE

x_resample, y_resample = SMOTE().fit_sample(train_df, y_train.values.ravel()) 

# checking the shape of x_resample and y_resample
print("Shape of x:", x_resample.shape)
print("Shape of y:", y_resample.shape)
#print(x_resample.columns)

Shape of x: (365086, 34)
Shape of y: (365086,)


In [38]:
# train and valid sets from train
"""from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_df, y_train, test_size = 0.2, random_state = 0)

# checking the shapes
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)"""

'from sklearn.model_selection import train_test_split\n\nx_train, x_valid, y_train, y_valid = train_test_split(train_df, y_train, test_size = 0.2, random_state = 0)\n\n# checking the shapes\nprint(x_train.shape)\nprint(y_train.shape)\nprint(x_valid.shape)\nprint(y_valid.shape)'

In [39]:
from sklearn import linear_model #for logistic regression
from sklearn.neural_network import MLPClassifier #for neural network
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, cross_val_predict, validation_curve 
#GridSearchCV is used to optimize parameters of the models used
#the other modules and functions 
from sklearn.ensemble import VotingClassifier #for creating ensembles of classifiers


In [40]:
# standardization

"""from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
test_df = sc.transform(test_df)"""

'from sklearn.preprocessing import StandardScaler\n\nsc = StandardScaler()\nx_train = sc.fit_transform(x_train)\nx_valid = sc.transform(x_valid)\ntest_df = sc.transform(test_df)'

In [41]:
# RANDOM FOREST CLASSIFIER
"""from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_rf = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, max_features=0.5, n_jobs=-1)
kfold = StratifiedKFold(n_splits=5, random_state=7)
cv_results = cross_val_score(model_rf, train_df, y_train, cv=kfold)
print (cv_results.mean()*100, "%")
model_rf.fit(x_train, y_train)

y_pred = model_rf.predict(x_valid)

print("Training Accuracy: ", model_rf.score(x_train, y_train))
print('Testing Accuarcy: ', model_rf.score(x_valid, y_valid))"""

'from sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import classification_report\n\nmodel_rf = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, max_features=0.5, n_jobs=-1)\nkfold = StratifiedKFold(n_splits=5, random_state=7)\ncv_results = cross_val_score(model_rf, train_df, y_train, cv=kfold)\nprint (cv_results.mean()*100, "%")\nmodel_rf.fit(x_train, y_train)\n\ny_pred = model_rf.predict(x_valid)\n\nprint("Training Accuracy: ", model_rf.score(x_train, y_train))\nprint(\'Testing Accuarcy: \', model_rf.score(x_valid, y_valid))'

In [42]:
"""from sklearn.model_selection import GridSearchCV
m = RandomForestClassifier()
parameters = [{'n_estimators': [100], 
               'min_samples_leaf': [3,5,7, 8, 9], 
               'max_features': ['sqrt', 'log2', 0.5]}
             ]
grid_search = GridSearchCV(estimator = m,
                           param_grid = parameters,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(x_train, y_)
best_accuracy = grid_search.best_score_#given by mean of 10 folds
best_parameters = grid_search.best_params_
print ("best accuracy is {}".format(best_accuracy))
print ("best parametrs are {}".format(best_parameters))"""

'from sklearn.model_selection import GridSearchCV\nm = RandomForestClassifier()\nparameters = [{\'n_estimators\': [100], \n               \'min_samples_leaf\': [3,5,7, 8, 9], \n               \'max_features\': [\'sqrt\', \'log2\', 0.5]}\n             ]\ngrid_search = GridSearchCV(estimator = m,\n                           param_grid = parameters,\n                           scoring = \'roc_auc\',\n                           cv = 10,\n                           n_jobs = -1)\ngrid_search = grid_search.fit(x_train, y_)\nbest_accuracy = grid_search.best_score_#given by mean of 10 folds\nbest_parameters = grid_search.best_params_\nprint ("best accuracy is {}".format(best_accuracy))\nprint ("best parametrs are {}".format(best_parameters))'

In [43]:
"""from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.001)
model_ada.fit(x_train, y_train)

y_pred = model_ada.predict(x_valid)

print("Training Accuracy: ", model_ada.score(x_train, y_train))
print('Testing Accuarcy: ', model_ada.score(x_valid, y_valid))"""

'from sklearn.ensemble import AdaBoostClassifier\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import classification_report\n\nmodel_ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.001)\nmodel_ada.fit(x_train, y_train)\n\ny_pred = model_ada.predict(x_valid)\n\nprint("Training Accuracy: ", model_ada.score(x_train, y_train))\nprint(\'Testing Accuarcy: \', model_ada.score(x_valid, y_valid))'

In [44]:
x = pd.DataFrame(x_resample, columns=train_df.columns)
y_resample

array([0., 1., 0., ..., 1., 1., 1.])

In [45]:
x.shape

(365086, 34)

In [46]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
#from sklearn.model_selection import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [47]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_xgb = XGBClassifier(n_estimators=120, learning_rate=1, n_jobs=-1,random_state=42)
#kfold = KFold(n_splits=5,random_state=7)
kfold = StratifiedKFold(n_splits=5, random_state=7)
cv_results = cross_val_score(model_xgb, x, y_resample, cv=kfold, scoring='roc_auc')
print (cv_results.mean()*100, "%")
"""model_xgb.fit(train_df, y_train)

y_pred = model_xgb.predict(x_valid)

print("Training Accuracy: ", model_xgb.score(x_train, y_train))
print('Testing Accuarcy: ', model_xgb.score(x_valid, y_valid))"""
# age score = 79.93900708025433 %
#age, Employmenttype, kfold score =80.%
#age, Employementtype, stratified fold = 80.62
#smote and droping min no. features(35) 88.65
# after log transformation of 7 features(Total group) 88.7837

88.8731496281571 %


'model_xgb.fit(train_df, y_train)\n\ny_pred = model_xgb.predict(x_valid)\n\nprint("Training Accuracy: ", model_xgb.score(x_train, y_train))\nprint(\'Testing Accuarcy: \', model_xgb.score(x_valid, y_valid))'

In [48]:
model_xgb.fit( x, y_resample)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=120,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [49]:
import pandas as pd
feature_importances = pd.DataFrame(model_xgb.feature_importances_,
                                   index = x.columns,
                                    columns=['importance']).sort_values('importance', 
                                                                        ascending=False)
feature_importances.head

<bound method NDFrame.head of                                      importance
major_city                             0.334299
Aadhar_flag                            0.187123
within.age.limit                       0.111054
NEW.ACCTS.IN.LAST.SIX.MONTHS           0.062415
Disbursal_month                        0.042155
manufacturer_id                        0.033433
loan_tenure_flag                       0.032342
VoterID_flag                           0.025000
NO.OF_INQUIRIES                        0.024574
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS    0.022878
Employment.Type                        0.015238
PAN_flag                               0.014753
TOTAL.ACTIVE.ACCTS                     0.013061
PERFORM_CNS.SCORE.DESCRIPTION          0.012245
Postal.zone                            0.011357
ltv                                    0.009041
PERFORM_CNS.SCORE                      0.007349
Age                                    0.007008
TOTAL.OVERDUE.ACCTS                    0.004991
branch_id 

In [50]:
"""from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_lgb = LGBMClassifier()
kfold = KFold(n_splits=5,random_state=7)
cv_results = cross_val_score(model_lgb, x_norm, y_train, cv=kfold)
print (cv_results.mean()*100, "%")
#model_lgb.fit(train_df, y_train)

#y_pred = model_lgb.predict(x_valid)

#print("Training Accuracy: ", model_lgb.score(x_train, y_train))
#print('Testing Accuarcy: ', model_lgb.score(x_valid, y_valid))

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-50-4ce56cfb2987>, line 14)

In [51]:
"""x_mean = train_df.mean()
x_std = train_df.std()
x_norm = (train_df - x_mean)/x_std
print (x_norm.shape)"""

'x_mean = train_df.mean()\nx_std = train_df.std()\nx_norm = (train_df - x_mean)/x_std\nprint (x_norm.shape)'

In [52]:
"""test_mean = test_df.mean()
test_std = test_df.std()
test_norm = (test_df - test_mean)/test_std
print (test_norm.shape)"""

'test_mean = test_df.mean()\ntest_std = test_df.std()\ntest_norm = (test_df - test_mean)/test_std\nprint (test_norm.shape)'

In [53]:
"""logreg = linear_model.LogisticRegression()
kfold = KFold(n_splits=5,random_state=7)
cv_results = cross_val_score(logreg, x_norm, y_train, cv=kfold)
print (cv_results.mean()*100, "%")"""

'logreg = linear_model.LogisticRegression()\nkfold = KFold(n_splits=5,random_state=7)\ncv_results = cross_val_score(logreg, x_norm, y_train, cv=kfold)\nprint (cv_results.mean()*100, "%")'

In [54]:
"""logreg = linear_model.LogisticRegression()
param_grid = {"C":[0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}
grid = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=kfold)
grid.fit(x_norm,y)
print (grid.best_estimator_)
print (grid.best_score_*100, "%")"""

'logreg = linear_model.LogisticRegression()\nparam_grid = {"C":[0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}\ngrid = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=kfold)\ngrid.fit(x_norm,y)\nprint (grid.best_estimator_)\nprint (grid.best_score_*100, "%")'

In [55]:
"""clf = MLPClassifier(solver='sgd',learning_rate='adaptive', random_state=1, activation='relu', hidden_layer_sizes=(15,),
                n_iter_no_change = 20, early_stopping=True)
kfold = KFold(n_splits=5,random_state=7)
cv_results = cross_val_score(clf, x_norm, y_train, cv=kfold)
print (cv_results.mean()*100, "%")"""

'clf = MLPClassifier(solver=\'sgd\',learning_rate=\'adaptive\', random_state=1, activation=\'relu\', hidden_layer_sizes=(15,),\n                n_iter_no_change = 20, early_stopping=True)\nkfold = KFold(n_splits=5,random_state=7)\ncv_results = cross_val_score(clf, x_norm, y_train, cv=kfold)\nprint (cv_results.mean()*100, "%")'

In [56]:
#clf.fit(x_norm, y_train)

In [57]:
#y_pred_rf = model_rf.predict(x_test)
#y_pred_ada = model_ada.predict(x_test)
y_pred_xgb = model_xgb.predict(test_df)
#y_pred_lgb = model_lgb.predict(test_df)
#y_pred_mpl = clf.predict(test_norm)

In [58]:
sub['loan_default']= y_pred_xgb
sub.to_csv("submission_xgb6.csv",index=False)
sub['loan_default'].value_counts()

0.0    111382
1.0      1010
Name: loan_default, dtype: int64

In [59]:
"""clf1 = linear_model.LogisticRegression()
clf2 = MLPClassifier(solver='lbfgs', alpha=1.0,hidden_layer_sizes=(15,), random_state=1, activation='logistic')
eclf = VotingClassifier(estimators=[('lr', clf1), ('nn', clf2)], voting='soft', weights=[1,2])
cv_results = cross_val_score(eclf, x_norm, y_train, cv=kfold)
print (cv_results.mean()*100, "%")"""

'clf1 = linear_model.LogisticRegression()\nclf2 = MLPClassifier(solver=\'lbfgs\', alpha=1.0,hidden_layer_sizes=(15,), random_state=1, activation=\'logistic\')\neclf = VotingClassifier(estimators=[(\'lr\', clf1), (\'nn\', clf2)], voting=\'soft\', weights=[1,2])\ncv_results = cross_val_score(eclf, x_norm, y_train, cv=kfold)\nprint (cv_results.mean()*100, "%")'